# 1. Setup Azure
Now that we've testing things locally, we're ready to use Azure to scale up our workload, automate and operationalize the job. 

First, we will walk through setting up credentials and settings for connecting to and creating resources in Azure. Then we will create an Azure blob storage which will be used throughout the tutorial as our object store. 

---

Import utilities.

In [2]:
import re
from dotenv import set_key, get_key, find_dotenv
%load_ext dotenv

In [4]:
env_path = find_dotenv()
if env_path=='':
    Path('.env').touch()
    env_path = find_dotenv()

## Set up your resource group
Make sure you've identified which subscription_id and region to use. Create a new resource group to contain all the resources that we create.

This section of the notebook will walk through setting up the resource group using the __az cli__.

Select which subscription you'd like to use for this project:
1. Print out all available subscriptions associated with your account
2. Set the Subscription ID you'd like to use as variable `subscription_id`
3. In the __az cli__, set the active subscription

In [1]:
!az account list -o table

Name                        CloudName    SubscriptionId                        State    IsDefault
--------------------------  -----------  ------------------------------------  -------  -----------
js_cloudai_subscription     AzureCloud   6dccbc1d-7b59-4ed6-bf8c-9747be083210  Enabled  False
Cosmos_WDG_Core_BnB_100348  AzureCloud   dae41bd3-9db4-4b9b-943e-832b57cac828  Enabled  False
1DS4ES-NonProd              AzureCloud   8c52214a-96c8-44e4-af24-31c54b965422  Enabled  False
1DS4ES-Prod                 AzureCloud   9a2ea167-0faf-4834-a0f7-cc08e844bf82  Enabled  False
Azure Cat E2E               AzureCloud   fc4ea3c9-1d30-4f18-b33b-7404e7da0123  Enabled  False
Core-ES-BLD                 AzureCloud   54e18c35-3863-4a17-8e52-b5aa1e65847e  Enabled  False
Batch-Portal-Test-In-Prod   AzureCloud   21abd678-18c5-4660-9fdd-8c5ba6b6fe1f  Enabled  False
Boston Team Danielle        AzureCloud   edf507a2-6235-46c5-b560-fd463ba2e771  Enabled  True
US Deployments              AzureCloud   75782cc3-a

In [ ]:
subscription_id = "<your-subscription>"

In [ ]:
!az account set -s $subscription_id

Choose a name for the resource group you'll be using for this project. You can think of resource groups as logical containers for the resources you'll create in this tutorial.

In [ ]:
resource_group = "<your-resource-group>"

Select the region you wish to deploy your resources in. 

NOTE: You can see a list of the regions under the key 'name' when running the command `az account list-locations`. 

NOTE: Not all regions support GPU enabled VMs. You can check [here](https://azure.microsoft.com/en-us/pricing/details/virtual-machines/linux/)

In [ ]:
region = "<selected-region>"

Create the resource group which we'll put our storage account and all other resources for this project into.

In [ ]:
!az group create -l $region -n $resource_group

Since we'll be using these settings throughout this tutorial, we'll also same them to the `.env` file.

In [ ]:
set_key(env_path, "SUBSCRIPTION_ID", subscription_id)
set_key(env_path, "RESOURCE_GROUP", resource_group)
set_key(env_path, "REGION", region)

## Create Azure blob storage
In this section of the notebook, we'll create an Azure blob storage that we'll use throughout the tutorial. This object store will be used to store input and output images as well as any supplementary data such as logs and other scripts that will be used in this workflow.

Create the storage account:
1. Select a name for your storage account and assign it to the variable `storage_account_name`
2. Use the __az cli__ to create the account

In [ ]:
storage_account_name = "<your-storage-account-name>"

In [ ]:
!az storage account create -n $storage_account_name -g $resource_group --query 'provisioningState'

Use the __az cli__ to grab the keys of the storage account that was just created. The `--quote '[0].value'` part of the command simply means to select the _value_ of the _zero-th indexed_ of the set of keys.

In [ ]:
key = !az storage account keys list --account-name $storage_account_name -g $resource_group --query '[0].value'

The stdout from the command above is stored in a string array of 1. Select the element in the array and ttrip opening and closing quotation marks.

In [ ]:
storage_account_key = str(key[0][1:-1]) # this is used to strip opening and closing quotation marks

The container in the storage account is a logical container for individual blobs. For this project, we'll store all content into a single container for ease-of-use.
1. Select a name for your storage container and assign it to the variable `azure_container_name`
2. Use the __az cli__ to create the container in the storage account

In [ ]:
storage_container_name = "aks"

In [ ]:
!az storage container create \
    --account-name $storage_account_name \
    --account-key $storage_account_key \
    --name $storage_container_name

Since we'll be using these settings throughout this tutorial, we'll also same them to the `.env` file.

In [ ]:
set_key(env_path, "STORAGE_ACCOUNT_NAME", storage_account_name)
set_key(env_path, "STORAGE_ACCOUNT_KEY", storage_account_key)
set_key(env_path, "STORAGE_CONTAINER_NAME", storage_container_name)

## Add Models to storage

In [5]:
!azcopy \
    --source models \
    --destination https://{get_key(env_path, "STORAGE_ACCOUNT_NAME")}.blob.core.windows.net/{get_key(env_path, "STORAGE_CONTAINER_NAME")}/models2 \
    --dest-key {get_key(env_path, "STORAGE_ACCOUNT_KEY")} \
    --recursive \
    --resume "."

=Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 2.428 MB; Average Speed:271.32 KB/s.                      Finished: 2 file(s), 25.714 MB; Average Speed:2.29 MB/s.                       Finished: 3 file(s), 25.714 MB; Average Speed:1.94 MB/s.                       Finished: 4 file(s), 25.714 MB; Average Speed:1.68 MB/s.                                                                                                      Finished 4 of total 4 file(s).
                                                                               [2018/11/08 02:46:15] Transfer summary:
-----------------
Total files transferred: 4
Transfer successfully:   4
Transfer skipped:        0
Transfer failed:         0
Elapsed tim

## Create Service Bus

Set the name of your namespace and queue.

In [ ]:
namespace = "<your-service-bus-namespace-name>"
queue = "<your-service-bus-queue-name"

Create the namespace.

In [ ]:
!az servicebus namespace create \
    --resource-group $resource_group \
    --name $namespace 

Create a service bus queue. Set the lock duration to 5 minutes. This means that the lock for each queue message will last for 5 minutes.

In [ ]:
!az servicebus queue create \
    --resource-group $resource_group \
    --namespace-name $namespace \
    --name $queue \
    --lock-duration PT5M

By default, your servicebus resource will come with an key-value "authorization rule" pair - its key name will have the value: "RootManageSharedAccessKey". The following command simply verifies that the key name for the "authorization rule" exists and is named "RootManageSharedAccessKey".

In [ ]:
!az servicebus namespace authorization-rule list \
    --resource-group $resource_group \
    --namespace-name $namespace \
    -o json --query "[0].name"

In [ ]:
sb_key_name = "RootManageSharedAccessKey"

Get the primary key value to "RootManageSharedAccessKey".

In [ ]:
sb_credentials = !az servicebus namespace authorization-rule keys list \
    --resource-group $resource_group \
    --namespace-name $namespace \
    --name $sb_key_name \
    -o json --query "primaryKey"

In [ ]:
sb_key_value = re.findall(r'"(.*?)"', str(sb_credentials))[0]

In [ ]:
set_key(env_path, "SB_SHARED_ACCESS_KEY_NAME", sb_key_name)
set_key(env_path, "SB_SHARED_ACCESS_KEY_VALUE", sb_key_value)
set_key(env_path, "SB_NAMESPACE", namespace)
set_key(env_path, "SB_QUEUE", queue)

---

## Conclusion
In this notebook, we've setup a service principal which gives us authentication to create resources in Azure. We've also create a storage account which will use throughout the rest of this tutorial. Next, we're going to [create our Batch AI cluster and run our work at scale in the cloud.](04_run_style_transfer_at_scale.ipynb)

In [ ]:
!cat .env